In [1]:
library(shiny)
library(plyr)
library(plotly)
library(ggmap)
library(tidyverse)

Loading required package: ggplot2

Attaching package: ‘plotly’

The following object is masked from ‘package:ggplot2’:

    last_plot

The following objects are masked from ‘package:plyr’:

    arrange, mutate, rename, summarise

The following object is masked from ‘package:stats’:

    filter

The following object is masked from ‘package:graphics’:

    layout

Google's Terms of Service: https://cloud.google.com/maps-platform/terms/.
Please cite ggmap if you use it! See citation("ggmap") for details.

Attaching package: ‘ggmap’

The following object is masked from ‘package:plotly’:

    wind

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ tibble  2.0.1     ✔ purrr   0.3.0
✔ tidyr   0.8.2     ✔ dplyr   0.7.8
✔ readr   1.3.1     ✔ stringr 1.3.1
✔ tibble  2.0.1     ✔ forcats 0.4.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::arrange()   masks plotly::arrange(), plyr::arrange()
✖ purrr::compact()   masks plyr

In [2]:
df <- read.csv("2011_us_ag_exports.csv", header = T)

#write.csv(df, '2011_us_ag_exports.csv')
#head(df)
df <- df[, c(1,2,3,4)]

chart5_data <- read.csv(file = 'StateDataforMap_2018-19week8.csv', header = T, stringsAsFactors = F)
#head(chart5_data)
chart5_data <- chart5_data[,-c(2,3)]

#https://stackoverflow.com/questions/42286072/extracting-numeric-values-from-mixed-variable-column-in-r
chart5_data$ACTIVITY.LEVEL <- sapply(strsplit(chart5_data$ACTIVITY.LEVEL, ' '), function(x) as.numeric(x[[2]]))

req_data <- left_join(chart5_data, df, by = c("STATENAME"="state"))

# give state boundaries a white border
l <- list(color = toRGB("white"), width = 2)

# specify some map projection/options
g <- list(
  scope = 'usa',
  projection = list(type = 'albers usa'),
  showlakes = TRUE,
  lakecolor = toRGB('white')
)

req_data$hover <- with(req_data, paste('Level ',ACTIVITY.LEVEL ))

cdcData <- plot_geo(req_data, locationmode = 'USA-states') %>%
  add_trace(
    z = ~ACTIVITY.LEVEL,text = ~hover,  locations = ~code,
    color = ~ACTIVITY.LEVEL, colors = 'Reds', legendgroup = ~ACTIVITY.LEVEL.LABEL
  ) %>%
  colorbar(title = "ACTIVITY LEVEL") %>%
  layout(
    title = 'CDC heatmap',
    geo = g
  )

Warning message:
“Column `STATENAME`/`state` joining character vector and factor, coercing into character vector”

In [3]:
#-------------------------------Twitter 1---------------------------------


##ETL
flu1 <- read.csv('flu1.csv', header = T, stringsAsFactors = F)
dim(flu1)
#flu1 <- rt

locations <- flu1$location
#View(table(locations))
#View(unique(locations))

temp1 <- as.vector(locations)
#https://stat545.com/block022_regular-expression.html
temp1 <- temp1[grepl(', [A-Z]{2}$',temp1)]
#View(temp1)

temp2 <- c() #appending to a list
for (i in temp1){
  temp2<-c(temp2,strsplit(i, ',')[[1]][2])
}

temp2
length(temp2)

tbl <- table(temp2)
population <- as.data.frame(tbl, stringsAsFactors = F)
population$temp2 <- trimws(population$temp2) #trimws works on entire column

tmp <- as.vector(locations)

temp3 <- c() #appending to a list
for (i in tmp){
  if (length(strsplit(i, ',')[[1]]) == 2){
    x <- strsplit(i, ',')[[1]][2]
    if ("USA" == trimws(x)){
      temp3<-c(temp3,strsplit(i, ',')[[1]][1])}
  }
}

tbl1 <- table(temp3)
population1 <- as.data.frame(tbl1, stringsAsFactors = F)
population1$temp3 <- trimws(population1$temp3) #trimws works on entire column


library(tidyverse)
library(usmap)

req_data <- merge(x = population, y = statepop, x.by = temp2, y.by = abbr, x.all = TRUE)
req_data <- filter(req_data, temp2 == abbr)
#View(req_data)
req_data <- req_data[, c(2,3,4,5)]
names(req_data)

req_data1 <- merge(x = population1, y = statepop, x.by = temp3, y.by = full, x.all = TRUE)
req_data1 <- filter(req_data1, temp3 == full)
#req_data <- req_data[, c(2,3,4,5)]
req_data1 <- req_data1[, c(2,3,4,5)]
names(req_data1) <- c('Freq_1', 'fips', 'abbr', 'full')

res <- merge(x = req_data,y = req_data1, all = T)
res$Freq[is.na(res$Freq)] <- 0
res$Freq_1[is.na(res$Freq_1)] <- 0
res$frequency <- res$Freq + res$Freq_1
names(res)
res <- res[, c(1,2,3,6)]
#nchar(req_data$temp2)
#nchar(trimws(req_data$temp2))
#View(res)
# heatmap <- plot_usmap(data = res, values = "frequency", lines = "black") +
#   #scale_fill_manual(values = c( "#CC0000", "#D73800", "#E27100", "#EEAA00", "#F9E200", "#E2F800", "#AAEA00", "#71DD00", "#38CF00", "#00C200")) +
#   theme(legend.position = "right") +
#   labs(fill = "Population")
#
# heatmap
##############################################################################################################################
#library(tidyverse)
df <- read.csv("2011_us_ag_exports.csv", header = T)
head(df)
#write.csv(df, '2011_us_ag_exports.csv')
#head(df)
df <- df[, c(2,3,4,5)]
names(df)
#chart5_data <- read.csv(file = 'StateDataforMap_2018-19week8.csv', header = T, stringsAsFactors = F)
#head(chart5_data)
chart5_data <- res
#names(res)
#chart5_data <- chart5_data[,-c(2,3)]

#https://stackoverflow.com/questions/42286072/extracting-numeric-values-from-mixed-variable-column-in-r
#chart5_data$ACTIVITY.LEVEL <- sapply(strsplit(chart5_data$ACTIVITY.LEVEL, ' '), function(x) as.numeric(x[[2]]))

req_data <- left_join(chart5_data, df, by = c("abbr"="code"))

# give state boundaries a white border
l <- list(color = toRGB("white"), width = 2)

# specify some map projection/options
g <- list(
  scope = 'usa',
  projection = list(type = 'albers usa'),
  showlakes = TRUE,
  lakecolor = toRGB('white')
)
names(req_data)
req_data$hover <- with(req_data, frequency)

twitter1 <- plot_geo(req_data, locationmode = 'USA-states') %>%
  add_trace(
    z = ~frequency,text = ~hover,  locations = ~abbr,
    color = ~frequency, colors = 'Reds'
  ) %>%
  colorbar(title = "ACTIVITY LEVEL") %>%
  layout(
    title = 'Twitter Flu heatmap',
    geo = g
  )


[1] 19793    88

[1] " AZ"                " NC"                " VA"               
   [4] " AR"                " OK"                " NC"               
   [7] " TN"                " TN"                " PA"               
  [10] " IL"                " TX"                " LA"               
  [13] " OR"                " NV"                " AL"               
  [16] " AL"                " NY"                " CA"               
  [19] " NY"                " GA"                " SC"               
  [22] " PA"                " OH"                " LA"               
  [25] " OR"                " SC"                " CA"               
  [28] " LA"                " WA"                " IL"               
  [31] " WI"                " OH"                " OH"               
  [34] " IN"                " AZ"                " IL"               
  [37] " OR"                " CT"                " WI"               
  [40] " IL"                " NC"                " OK"               
  [43] " NC"                " NC"                " GA"               
  [46] " GA"                " MN"                " TX"               
  [49] " MS"                " NC"                " NC"               
  [52] " VA"                " TX"                " CA"               
  [55] " FL"                " NC"                " GA"               
  [58] " NY"                " MN"                " NJ"               
  [61] " DC"                " TX"                " TX"               
  [64] " CA"                " OH"                " OH"               
  [67] " CA"                " TX"                " NC"               
  [70] " OH"                " OH"                " NM"               
  [73] " TX"                " KS"                " UT"               
  [76] " KY"                " FL"                " OH"               
  [79] " LA"                " NC"                " FL"               
  [82] " MN"                " IN"                " CA"               
  [85] " CT"                " ID"                " DE"               
  [88] " OH"                " CA"                " CA"               
  [91] " FL"                " IA"                " GA"               
  [94] " GA"                " GA"                " GA"               
  [97] " OR"                " OR"                " TX"               
 [100] " CA"                " AZ"                " NE"               
 [103] " WI"                " MD"                " CO"               
 [106] " CO"                " TN"                " IN"               
 [109] " CA"                " CA"                " CA"               
 [112] " AZ"                " MO"                " FL"               
 [115] " FL"                " CO"                " LA"               
 [118] " WI"                " WI"                " MI"               
 [121] " FL"                " TX"                " FL"               
 [124] " CO"                " IN"                " GA"               
 [127] " CA"                " DC"                " TX"               
 [130] " CA"                " TX"                " MO"               
 [133] " MO"                " VA"                " WV"               
 [136] " GA"                " CA"                " GA"               
 [139] " CA"                " PA"                " ME"               
 [142] " MN"                " TX"                " IL"               
 [145] " DC"                " NY"                " NY"               
 [148] " NC"                " DC"                " NC"               
 [151] " NC"                " NC"                " NC"               
 [154] " AZ"                " OH"                " OH"               
 [157] " NV"                " CA"                " NV"               
 [160] " LA"                " NY"                " TX"               
 [163] " NV"                " MA"                " IL"               
 [166] " NC"                " VA"                " IL"               
 [169] " OH"                " PA"                " DC"               
 [172] " MD" 

[1] 8468

[1] "Freq" "fips" "abbr" "full"

[1] "fips"      "abbr"      "full"      "Freq"      "Freq_1"    "frequency"

X,code,state,category,total.exports,beef,pork,poultry,dairy,fruits.fresh,fruits.proc,total.fruits,veggies.fresh,veggies.proc,total.veggies,corn,wheat,cotton
1,AL,Alabama,state,1390.63,34.4,10.6,481.0,4.06,8.0,17.1,25.11,5.5,8.9,14.33,34.9,70.0,317.61
2,AK,Alaska,state,13.31,0.2,0.1,0.0,0.19,0.0,0.0,0.00,0.6,1.0,1.56,0.0,0.0,0.00
3,AZ,Arizona,state,1463.17,71.3,17.9,0.0,105.48,19.3,41.0,60.27,147.5,239.4,386.91,7.3,48.7,423.95
4,AR,Arkansas,state,3586.02,53.2,29.4,562.9,3.53,2.2,4.7,6.88,4.4,7.1,11.45,69.5,114.5,665.44
5,CA,California,state,16472.88,228.7,11.1,225.4,929.95,2791.8,5944.6,8736.40,803.2,1303.5,2106.79,34.6,249.3,1064.95
6,CO,Colorado,state,1851.33,261.4,66.0,14.0,71.94,5.7,12.2,17.99,45.1,73.2,118.27,183.2,400.5,0.00


[1] "code"          "state"         "category"      "total.exports"

Warning message:
“Column `abbr`/`code` joining character vector and factor, coercing into character vector”

[1] "fips"          "abbr"          "full"          "frequency"    
[5] "state"         "category"      "total.exports"

In [4]:
#----------------------------Twitter 2 ----------------------------------

##ETL
flu1 <- read.csv('flu_18k_1.csv', header = T, stringsAsFactors = F)
dim(flu1)
#flu1 <- rt

locations <- flu1$location
#View(table(locations))
#View(unique(locations))

temp1 <- as.vector(locations)
#https://stat545.com/block022_regular-expression.html
temp1 <- temp1[grepl(', [A-Z]{2}$',temp1)]
#View(temp1)

temp2 <- c() #appending to a list
for (i in temp1){
  temp2<-c(temp2,strsplit(i, ',')[[1]][2])
}

temp2
length(temp2)

tbl <- table(temp2)
population <- as.data.frame(tbl, stringsAsFactors = F)
population$temp2 <- trimws(population$temp2) #trimws works on entire column

tmp <- as.vector(locations)

temp3 <- c() #appending to a list
for (i in tmp){
  if (length(strsplit(i, ',')[[1]]) == 2){
    x <- strsplit(i, ',')[[1]][2]
    if ("USA" == trimws(x)){
      temp3<-c(temp3,strsplit(i, ',')[[1]][1])}
  }
}

tbl1 <- table(temp3)
population1 <- as.data.frame(tbl1, stringsAsFactors = F)
population1$temp3 <- trimws(population1$temp3) #trimws works on entire column


library(tidyverse)
library(usmap)

req_data <- merge(x = population, y = statepop, x.by = temp2, y.by = abbr, x.all = TRUE)
req_data <- filter(req_data, temp2 == abbr)
#View(req_data)
req_data <- req_data[, c(2,3,4,5)]
names(req_data)

req_data1 <- merge(x = population1, y = statepop, x.by = temp3, y.by = full, x.all = TRUE)
req_data1 <- filter(req_data1, temp3 == full)
#req_data <- req_data[, c(2,3,4,5)]
req_data1 <- req_data1[, c(2,3,4,5)]
names(req_data1) <- c('Freq_1', 'fips', 'abbr', 'full')

res <- merge(x = req_data,y = req_data1, all = T)
res$Freq[is.na(res$Freq)] <- 0
res$Freq_1[is.na(res$Freq_1)] <- 0
res$frequency <- res$Freq + res$Freq_1
names(res)
res <- res[, c(1,2,3,6)]
#nchar(req_data$temp2)
#nchar(trimws(req_data$temp2))
#View(res)
# heatmap <- plot_usmap(data = res, values = "frequency", lines = "black") +
#   #scale_fill_manual(values = c( "#CC0000", "#D73800", "#E27100", "#EEAA00", "#F9E200", "#E2F800", "#AAEA00", "#71DD00", "#38CF00", "#00C200")) +
#   theme(legend.position = "right") +
#   labs(fill = "Population")
#
# heatmap
##############################################################################################################################
#library(tidyverse)
df <- read.csv("2011_us_ag_exports.csv", header = T)
head(df)
#write.csv(df, '2011_us_ag_exports.csv')
#head(df)
df <- df[, c(2,3,4,5)]
names(df)
#chart5_data <- read.csv(file = 'StateDataforMap_2018-19week8.csv', header = T, stringsAsFactors = F)
#head(chart5_data)
chart5_data <- res
#names(res)
#chart5_data <- chart5_data[,-c(2,3)]

#https://stackoverflow.com/questions/42286072/extracting-numeric-values-from-mixed-variable-column-in-r
#chart5_data$ACTIVITY.LEVEL <- sapply(strsplit(chart5_data$ACTIVITY.LEVEL, ' '), function(x) as.numeric(x[[2]]))

req_data <- left_join(chart5_data, df, by = c("abbr"="code"))

# give state boundaries a white border
l <- list(color = toRGB("white"), width = 2)

# specify some map projection/options
g <- list(
  scope = 'usa',
  projection = list(type = 'albers usa'),
  showlakes = TRUE,
  lakecolor = toRGB('white')
)
names(req_data)
req_data$hover <- with(req_data, frequency)

twitter2 <- plot_geo(req_data, locationmode = 'USA-states') %>%
  add_trace(
    z = ~frequency,text = ~hover,  locations = ~abbr,
    color = ~frequency, colors = 'Reds'
  ) %>%
  colorbar(title = "ACTIVITY LEVEL") %>%
  layout(
    title = 'Twitter heatmap',
    geo = g
  )

[1] 16681    88

[1] " FL"                " PA"                " CA"               
   [4] " MA"                " CO"                " IN"               
   [7] " KS"                " MN"                " CA"               
  [10] " NY"                " NY"                " NY"               
  [13] " NY"                " NY"                " TX"               
  [16] " MT"                " NY"                " NC"               
  [19] " MA"                " MN"                " TN"               
  [22] " DC"                " DC"                " DC"               
  [25] " MA"                " TX"                " TX"               
  [28] " NE"                " NE"                " NE"               
  [31] " NE"                " NY"                " NY"               
  [34] " CO"                " MA"                " UT"               
  [37] " CA"                " CA"                " CA"               
  [40] " IL"                " CA"                " DC"               
  [43] " NY"                " NY"                " NY"               
  [46] " NY"                " NY"                " NY"               
  [49] " NY"                " NY"                " NY"               
  [52] " NY"                " NY"                " NY"               
  [55] " NY"                " NY"                " NY"               
  [58] " NY"                " NY"                " NY"               
  [61] " NY"                " NY"                " NY"               
  [64] " NY"                " NY"                " NY"               
  [67] " NY"                " NY"                " NY"               
  [70] " NY"                " NY"                " NY"               
  [73] " NY"                " NY"                " NY"               
  [76] " DC"                " DC"                " DC"               
  [79] " DC"                " DC"                " CA"               
  [82] " CA"                " NE"                " UT"               
  [85] " NY"                " CA"                " NY"               
  [88] " NV"                " NV"                " NC"               
  [91] " UT"                " UT"                " UT"               
  [94] " UT"                " UT"                " OR"               
  [97] " CA"                " NY"                " DC"               
 [100] " GA"                " GA"                " GA"               
 [103] " GA"                " GA"                " GA"               
 [106] " GA"                " GA"                " NY"               
 [109] " NY"                " CO"                " VA"               
 [112] " OH"                " OH"                " FL"               
 [115] " NM"                " OH"                " PA"               
 [118] " OH"                " OH"                " MO"               
 [121] " MI"                " AZ"                " NE"               
 [124] " NE"                " MN"                " DC"               
 [127] " WA"                " NC"                " NY"               
 [130] " CA"                " NC"                " OH"               
 [133] " OH"                " MA"                " WA"               
 [136] " TX"                " DC"                " DC"               
 [139] " DC"                " DC"                " DC"               
 [142] " WI"                " WI"                " WI"               
 [145] " CA"                " CA"                " OR"               
 [148] " TX"                " PA"                " OH"               
 [151] " MO"                " MO"                " MO"               
 [154] " MO"                " CT"                " CT"               
 [157] " LA"                " CA"                " NY"               
 [160] " ND"                " CA"                " CA"               
 [163] " TX"                " TX"                " OH"               
 [166] " DC"                " OH"                " IN"               
 [169] " CA"                " NC"                " GA"               
 [172] " GA" 

[1] 7329

[1] "Freq" "fips" "abbr" "full"

[1] "fips"      "abbr"      "full"      "Freq"      "Freq_1"    "frequency"

X,code,state,category,total.exports,beef,pork,poultry,dairy,fruits.fresh,fruits.proc,total.fruits,veggies.fresh,veggies.proc,total.veggies,corn,wheat,cotton
1,AL,Alabama,state,1390.63,34.4,10.6,481.0,4.06,8.0,17.1,25.11,5.5,8.9,14.33,34.9,70.0,317.61
2,AK,Alaska,state,13.31,0.2,0.1,0.0,0.19,0.0,0.0,0.00,0.6,1.0,1.56,0.0,0.0,0.00
3,AZ,Arizona,state,1463.17,71.3,17.9,0.0,105.48,19.3,41.0,60.27,147.5,239.4,386.91,7.3,48.7,423.95
4,AR,Arkansas,state,3586.02,53.2,29.4,562.9,3.53,2.2,4.7,6.88,4.4,7.1,11.45,69.5,114.5,665.44
5,CA,California,state,16472.88,228.7,11.1,225.4,929.95,2791.8,5944.6,8736.40,803.2,1303.5,2106.79,34.6,249.3,1064.95
6,CO,Colorado,state,1851.33,261.4,66.0,14.0,71.94,5.7,12.2,17.99,45.1,73.2,118.27,183.2,400.5,0.00


[1] "code"          "state"         "category"      "total.exports"

Warning message:
“Column `abbr`/`code` joining character vector and factor, coercing into character vector”

[1] "fips"          "abbr"          "full"          "frequency"    
[5] "state"         "category"      "total.exports"

In [ ]:
#-----------Done------------------

ui <- fluidPage(

  headerPanel("Heat map comparison"),

  sidebarPanel(
    selectInput("plot", "Maps:", 
                c("CDC vs Twitter Flu" = "op1",
                  "Twitter Flu vs Twitter" = "op2",
                  "CDC vs Twitter" = "op3"))),

  mainPanel(plotlyOutput("plot1"), plotlyOutput("plot2"))

)



server <- function(input, output) {
  output$plot1 <- renderPlotly({
    
    if (input$plot=='op2') {
      twitter1;
    } else {
      cdcData
    }
    
  })
  output$plot2 <- renderPlotly({
    if (input$plot=='op1') {
      twitter1;
    } else {
      twitter2;
    }
  })
}


shinyApp(ui, server)


Listening on http://127.0.0.1:7842


### CDC  - Data gathered from the CDC site
### Twitter Flu - Twitter location data acquired using search query 'flu'
### Twitter - Twitter location data acquired using additional keywords, which are available in the code

### CDC map shows that central and south east USA have relatively higher activity levels
### Twitter Flu map shows that Texas and California have max reports in twitter regarding flu
### Twiiter map shows that, in general, flu related activities are reported mostly from California  